# Python Multi-threading and Concurrency

#### 1. Write a Python program to create multiple threads and print their names.

In [1]:
import threading
def print_thread_name():
    print(f"Current thread:{threading.current_thread().name}")
threads = []
for i in range(5):
    thread = threading.Thread(target = print_thread_name, name = f"thread-{i}")
    threads.append(thread)
    thread.start()

for i in range(len(threads)):
    threads[i].join()

Current thread:thread-0
Current thread:thread-1
Current thread:thread-2
Current thread:thread-3
Current thread:thread-4


#### 2. Write a Python program to download multiple files concurrently using threads.

In [3]:
import threading
import requests

def download_file(url, filename):
    try:
        response = requests.get(url)
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded {filename} successfully!")
    except Exception as e:
        print(f"Failed to download {filename}: {e}")

def main():
    urls = [
        ("https://example.com/file1.jpg", "file1.jpg"),
        ("https://example.com/file2.jpg", "file2.jpg"),
        ("https://example.com/file3.jpg", "file3.jpg")
    ]
    
    threads = []
    
    for url, filename in urls:
        thread = threading.Thread(target=download_file, args=(url, filename))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()


Downloaded file1.jpg successfully!
Downloaded file2.jpg successfully!
Downloaded file3.jpg successfully!


#### 3. Write a Python program that creates two threads to find and print even and odd numbers from 30 to 50.

In [5]:
import threading
def print_even_numbers():
    arr = [num for num in range(30, 51) if num % 2 == 0]
    print(arr)
def print_odd_numbers():
    arr = [num for num in range(30, 51) if num % 2 != 0]
    print(arr)
def main():
    thread_even = threading.Thread(target = print_even_numbers)
    thread_odd = threading.Thread(target = print_odd_numbers)

    thread_even.start()
    thread_odd.start()
    thread_even.join()
    thread_odd.join()
if __name__ == "__main__":
    main()

[30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50]
[31, 33, 35, 37, 39, 41, 43, 45, 47, 49]


#### 4. Write a Python program to calculate the factorial of a number using multiple threads.

## Solution 1 (Explicitly creating and starting the threads) ##

In [21]:
import threading
from math import ceil

# Function to calculate partial factorial
def partial_factorial(start, end, result, index):
    product = 1
    for i in range(start, end + 1):
        product *= i
    result[index] = product

# Function to calculate factorial using multiple threads
def multi_thread_factorial(n, num_threads):
    # Create a list to store the partial results from threads
    result = [None] * num_threads

    # Calculate the range of numbers each thread will handle
    chunk_size = ceil(n / num_threads)

    # Create threads
    threads = []
    for i in range(num_threads):
        start = i * chunk_size + 1
        # Ensure the last thread covers the remainder if n is not divisible by num_threads
        end = min((i + 1) * chunk_size, n)
        thread = threading.Thread(target=partial_factorial, args=(start, end, result, i))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Multiply all partial results to get the final factorial
    final_result = 1
    for res in result:
        final_result *= res

    return final_result

# Example usage
n = 100  # Factorial of 10
num_threads = 3  # Number of threads

factorial_result = multi_thread_factorial(n, num_threads)
print(f"The factorial of {n} is {factorial_result}")
print("hello")




# -------------------------------------------------------------------------------------------
# Here queue is not used, as each thread is peroforming task on the uniform chunk of data and not the custom range
# additionally data being generated is stored inside of the result array at the particular index, so it is independent, so no need to use lock aswell

The factorial of 100 is 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
hello


## Solution 2 (thread creation, starting and joining handled by threadpool executor) ## 

In [29]:
import threading
from concurrent.futures import ThreadPoolExecutor
from math import ceil

# Function to calculate partial factorial
def partial_factorial(start, end):
    product = 1
    for i in range(start, end + 1):
        product *= i
    return product

# Function to calculate factorial using ThreadPoolExecutor
def multi_thread_factorial(n, num_threads):
    # Calculate the range of numbers each thread will handle
    chunk_size = ceil(n / num_threads)
    
    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit tasks to the executor
        futures = []
        for i in range(num_threads):
            start = i * chunk_size + 1
            end = min((i + 1) * chunk_size, n)
            futures.append(executor.submit(partial_factorial, start, end))
        
        # Collect all results from the futures
        final_result = 1
        for future in futures:
            final_result *= future.result()
    
    return final_result

# Example usage
n = 10  # Factorial of 10
num_threads = 3  # Number of threads

factorial_result = multi_thread_factorial(n, num_threads)
print(f"The factorial of {n} is {factorial_result}")


# ------------------------------------------------------
# by making use of threadpoolexecutor, the there is no need to manage another result array
# as the result being generated can be accessed calling the result method on the objects stored in future arrayy

The factorial of 10 is 3628800


In [9]:
import threading
from queue import Queue

def worker(q, lock):
    while True:
        item = None
        with lock:  # Ensure that only one thread accesses the queue at a time
            item = q.get()

        print("item", item)
        
        if item is None:  # Sentinel to terminate the thread
            with lock:  # Acquire lock before marking sentinel task as done
                q.task_done()  # Mark the sentinel as done
            break
        
        print(f"{threading.current_thread().name} processing {item}")
        
        with lock:  # Acquire lock before marking task as done
            q.task_done()

# Create a queue and add tasks
q = Queue()  # Correct import usage
for i in range(5):  # Add actual tasks
    q.put(i)

# Add sentinel values for each thread
num_threads = 3
for _ in range(num_threads):
    q.put(None)

# Create a lock
lock = threading.Lock()

# Start worker threads
threads = []
for i in range(num_threads):
    thread = threading.Thread(target=worker, args=(q, lock), name=f"Worker-{i}")
    thread.start()
    threads.append(thread)

q.join()  # Wait until all tasks (excluding sentinels) are processed

# Join threads
for thread in threads:
    thread.join()

print("All tasks processed, and threads terminated.")

item 0
Worker-0 processing 0
item 1
Worker-0 processing 1
item 2
Worker-0 processing 2
item 3
Worker-0 processing 3
item 4
Worker-0 processing 4
item None
item None
item None
All tasks processed, and threads terminated.


In [43]:
import threading
from queue import Queue

def worker(q, lock):
    while True:
        with lock:  # Ensure that only one thread accesses the queue at a time
            item = q.get()

        print("item", item)
        
        if item is None:  # Sentinel to terminate the thread
            with lock:  # Acquire lock before marking sentinel task as done
                q.task_done()  # Mark the sentinel as done
            break
        
        print(f"{threading.current_thread().name} processing {item}")
        
        with lock:  # Acquire lock before marking task as done
            q.task_done()

# Create a queue and add tasks
q = Queue()  # Correct import usage
for i in range(5):  # Add actual tasks
    q.put(i)

# Add sentinel values for each thread
num_threads = 3
for _ in range(num_threads):
    q.put(None)

# Create a lock
lock = threading.Lock()

# Start worker threads
threads = []
for i in range(num_threads):
    thread = threading.Thread(target=worker, args=(q, lock), name=f"Worker-{i}")
    thread.start()
    threads.append(thread)

q.join()  # Wait until all tasks (excluding sentinels) are processed

# Join threads
for thread in threads:
    thread.join()

print("All tasks processed, and threads terminated.")

item 0
Worker-0 processing 0
item 1
Worker-0 processing 1
item 2
Worker-0 processing 2
item 3
Worker-0 processing 3
item 4
Worker-0 processing 4
item None
item None
item None
All tasks processed, and threads terminated.


#### 7. Python Multi-threading and concurrency: Concurrent HTTP requests with threads

In [36]:
import requests
from concurrent.futures import ThreadPoolExecutor

def make_request(url):
    response = requests.get(url)
    print(f"Response from {url}: {response.status_code}") 

urls = [
    "https://www.example.com",
    "https://www.google.com",
    "https://www.wikipedia.org",
    "https://www.python.org"
]

with ThreadPoolExecutor() as executor:
    for url in urls:
        executor.submit(make_request, url)

Response from https://www.example.com: 200
Response from https://www.python.org: 200
Response from https://www.wikipedia.org: 200
Response from https://www.google.com: 200


# Python Asynchronous Programming Exercises and Solutions

#### 1. Write a Python program that creates an asynchronous function to print "Python Exercises!" with a two second delay.

In [60]:
import asyncio

async def print_message():
    await asyncio.sleep(2)
    print("Python Exercises!")

async def main():
    await print_message()

# asyncio.run(main()) this requires of creating another event loop, so it gives error in jupyter
await main() # this does not require of creating new event loop, so this is used to coroutine in a jupter environment


Python Exercises!


#### 2. Write a Python program that creates three asynchronous functions and displays their respective names with different delays (1 second, 2 seconds, and 3 seconds).

In [67]:

import asyncio
async def task_1():
    await asyncio.sleep(1)
    print(f"{task_1.__name__} completed")
async def task_2():
    await asyncio.sleep(2)
    print(f"{task_2.__name__} completed")
async def task_3():
    await asyncio.sleep(3)
    print(f"{task_3.__name__} completed")
async def main():

    # method-1: using create_task()
    # t1 = asyncio.create_task(task_1())
    # t2 = asyncio.create_task(task_2())
    # t3 = asyncio.create_task(task_3())

    # await t1
    # await t2
    # await t3
    await asyncio.gather(
        task_1(),
        task_2(),
        task_3()
    )
# asyncio.run(main())
await main()

task_1 completed
task_2 completed
task_3 completed


#### 3. Write a Python program that creates an asyncio event loop and runs a coroutine that prints numbers from 1 to 7 with a delay of 1 second each.

In [70]:
import asyncio
async def fun():
    for i in range(1, 8):
        print(i)
        await asyncio.sleep(1)
async def main():
    await fun()

await main()

1
2
3
4
5
6
7


#### 4. Write a Python program that implements a coroutine to fetch data from two different URLs simultaneously using the "aiohttp" library.

In [77]:
import aiohttp
import asyncio

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.text()
            print(f"Data from {url}: {data}...")  # Print first 100 characters of the response

async def main():
    url1 = 'https://jsonplaceholder.typicode.com/todos/1'  # Example URL 1
    url2 = 'https://jsonplaceholder.typicode.com/todos/2'  # Example URL 2
    await asyncio.gather(fetch_data(url1), fetch_data(url2))

await (main())


Data from https://jsonplaceholder.typicode.com/todos/1: {
  "userId": 1,
  "id": 1,
  "title": "delectus aut autem",
  "completed": false
}...
Data from https://jsonplaceholder.typicode.com/todos/2: {
  "userId": 1,
  "id": 2,
  "title": "quis ut nam facilis et officia qui",
  "completed": false
}...
